In [1]:
import pandas as pd

xls = pd.ExcelFile('companies_new.xlsx')
sheets = xls.sheet_names
sheet_to_df_map = {}
for sheet in sheets:
    sheet_to_df_map[sheet] = xls.parse(sheet)
    

df = pd.concat(sheet_to_df_map.values(), ignore_index=True)

In [2]:
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

In [3]:
df['Date'] = df['Date'].astype('datetime64[ns]')

In [4]:
df['COMP'].value_counts()

UNILEVER            1135
DEUTSCHEBANK        1127
HENKEL              1127
VOLKSWAGEN          1127
BMW                 1127
DELIVERY HERO       1127
ZALANDO             1127
LUFTHANSA           1127
HELLOFRESH          1117
DEUTSCHE TELEKOM    1116
ALLIANZ             1116
HUGO BOSS           1116
SAP                 1116
SIEMENS              992
BASF                 886
Name: COMP, dtype: int64

In [5]:
news = pd.read_csv('news_data.csv')

In [6]:
news = news[['just_date', 'message', 'COMP']].rename({'just_date': 'Date'}, axis = 'columns')

In [7]:
news['Date'] = news['Date'].astype('datetime64[ns]')

In [8]:
def func(x):
    res = x.upper() 
    if res == 'ZAL':
        return 'ZALANDO'
    else:
        return res

In [9]:
news['COMP'] = news['COMP'].apply(lambda x: func(x))

In [10]:
result = pd.merge(df, news, how="left", on=["COMP", "Date"])

In [11]:
result.message.notna().sum()

3661

In [14]:
result

,COMP,Date,Open,High,Low,Close,Volume,message
0,ALLIANZ,2019-02-22,79.96,81.35,79.80,81.23,423912,NaN
1,ALLIANZ,2019-02-25,81.76,82.67,81.72,82.41,540216,NaN
2,ALLIANZ,2019-02-26,82.25,83.16,82.16,82.68,327024,kurz vor dem gipfeltreffen: alle blicke sind a...
3,ALLIANZ,2019-02-27,82.43,83.53,82.07,83.27,366054,NaN
4,ALLIANZ,2019-02-28,82.05,82.06,81.07,81.81,519239,kommentar: die allianz der erzrivalen daimler ...
...,...,...,...,...,...,...,...,...
17655,ZALANDO,2023-07-24,30.75,31.25,30.28,30.82,1002680,NaN
17656,ZALANDO,2023-07-25,30.75,31.58,30.75,31.35,1146526,NaN
17657,ZALANDO,2023-07-26,31.05,31.85,30.45,30.72,825826,NaN
17658,ZALANDO,2023-07-27,30.45,32.17,30.45,31.35,980111,NaN


In [13]:
result.to_csv('final.csv')

In [15]:
result['COMP'].value_counts()

VOLKSWAGEN          1542
LUFTHANSA           1453
DEUTSCHEBANK        1357
ALLIANZ             1169
BMW                 1147
UNILEVER            1139
HENKEL              1133
ZALANDO             1133
DEUTSCHE TELEKOM    1131
DELIVERY HERO       1129
SAP                 1122
HELLOFRESH          1118
HUGO BOSS           1116
SIEMENS             1071
BASF                 900
Name: COMP, dtype: int64